In [1]:
import tensorflow as tf
import numpy as np
import pymysql

input_vec_size = lstm_size = 7
time_step_size = 60
label_size = 3
evaluate_size = 2
lstm_depth = 4

total_size = 60000
batch_size = 15000
test_size = total_size - batch_size
conn = pymysql.connect(host='192.168.1.210',
                                          user='root',
                                          password='1234',
                                          db='data',
                                          charset='utf8mb4')


def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [2]:
def model(code, X, W, B, lstm_size):
    XT = tf.transpose(X, [1, 0, 2]) 
    XR = tf.reshape(XT, [-1, lstm_size])
    X_split = tf.split(0, time_step_size, XR)
    with tf.variable_scope(code, reuse=False):
        cell = tf.nn.rnn_cell.GRUCell(lstm_size)
        cell = tf.nn.rnn_cell.DropoutWrapper(cell = cell, output_keep_prob = 0.5)
        cell = tf.nn.rnn_cell.MultiRNNCell([cell] * lstm_depth, state_is_tuple = True)

    # Get lstm cell output, time_step_size (60) arrays with lstm_size output: (batch_size, lstm_size)
    outputs, _states = tf.nn.rnn(cell, X_split, dtype=tf.float32)

    # Linear activation
    # Get the last output
    return tf.matmul(outputs[-1], W) + B, cell.state_size # State size to initialize the stat


In [3]:
def get_codedates(code):
    code_dates = list()
    cursor = conn.cursor()
    cursor.execute("SELECT date FROM data.daily_stock WHERE code = %s ORDER BY date", (code))
    dates = cursor.fetchall()
    for date in dates:
        code_dates.append((code, date[0]))    
    return code_dates

In [4]:
def read_series_datas(conn, code_dates):
    X = list()
    Y = list()
    for code_date in code_dates:
        cursor = conn.cursor()        
        cursor.execute("SELECT open, high, low, close, volume, hold_foreign, st_purchase_inst FROM data.daily_stock WHERE code = %s AND date >= %s ORDER BY date LIMIT %s", 
                       (code_date[0], code_date[1], time_step_size + evaluate_size))
        items = cursor.fetchall()        
        if len(items) < (evaluate_size + time_step_size):
            break
        X.append(np.array(items[:time_step_size]))

        st_purchase_inst = items[-(evaluate_size + 1)][6]
        if st_purchase_inst == 0:
            continue
        for i in range(evaluate_size, len(items) - evaluate_size):
            eval_inst = items[i][6]
            eval_bef = items[evaluate_size-i][6]
            if eval_bef < eval_inst:
                eval_bef = eval_inst           
        
        if (eval_bef - st_purchase_inst) / st_purchase_inst < -0.02: #percent ? cnt ? 
            Y.append((0., 0., 1.))
        elif (eval_bef - st_purchase_inst) / st_purchase_inst > 0.04:
            Y.append((1., 0., 0.))
        else:
            Y.append((0., 1., 0.))


    arrX = np.array(X)    
    meanX = np.mean(arrX, axis = 0)
    stdX = np.std(arrX, axis = 0)
    norX = (arrX - meanX) / stdX
    norY = np.array(Y)
    return norX, norY


In [5]:
def read_datas(code_dates):    
    np.random.seed()
    np.random.shuffle(code_dates)

    trX = list()
    trY = list()
    trX, trY = read_series_datas(conn, code_dates)
    teX, teY = read_series_datas(conn, code_dates)

    return trX, trY, teX, teY

In [6]:
def get_codes():
    cursor = conn.cursor()
    cursor.execute("SELECT DISTINCT code FROM data.daily_stock LIMIT 100")
    codes = cursor.fetchall()
    return codes

In [ ]:

for code in get_codes():
    tf.reset_default_graph()
    code_dates = get_codedates(code[0])
    trX, trY, teX, teY = read_datas(code_dates)
    if (len(trX) == 0):
        continue

    X = tf.placeholder(tf.float32, [None, time_step_size, input_vec_size])
    Y = tf.placeholder(tf.float32, [None, label_size])

    # get lstm_size and output 3 labels
    W = init_weights([lstm_size, label_size])
    B = init_weights([label_size])

    py_x, state_size = model(code[0], X, W, B, lstm_size)

    loss = tf.nn.softmax_cross_entropy_with_logits(py_x, Y)
    cost = tf.reduce_mean(loss)
    train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
    predict_op = tf.argmax(py_x, 1)

    # Launch the graph in a session
    with tf.Session() as sess:
        # you need to initialize all variables
        tf.global_variables_initializer().run()

        for i in range(100):
            for start, end in zip(range(0, len(trX), batch_size), range(batch_size, len(trX)+1, batch_size)):
                sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end]})

            test_indices = np.arange(len(teX))
            org = teY[test_indices]
            res = sess.run(predict_op, feed_dict={X: teX[test_indices], Y: teY[test_indices]})

            print(code[0], i, np.mean(np.argmax(org, axis=1) == res))


A000030 0 0.0
A000030 1 0.0
A000030 2 0.0
A000030 3 0.0
A000030 4 0.333333333333
A000030 5 0.0
A000030 6 0.333333333333
A000030 7 0.333333333333
A000030 8 0.0
A000030 9 0.0
A000030 10 0.333333333333
A000030 11 0.0
A000030 12 0.0
A000030 13 0.333333333333
A000030 14 0.333333333333
A000030 15 0.0
A000030 16 0.0
A000030 17 0.333333333333
A000030 18 0.333333333333
A000030 19 0.0
A000030 20 0.0
A000030 21 0.0
A000030 22 0.333333333333
A000030 23 0.0
A000030 24 0.333333333333
A000030 25 0.0
A000030 26 0.0
A000030 27 0.0
A000030 28 0.0
A000030 29 0.333333333333
A000030 30 0.0
A000030 31 0.333333333333
A000030 32 0.333333333333
A000030 33 0.0
A000030 34 0.0
A000030 35 0.333333333333
A000030 36 0.0
A000030 37 0.333333333333
A000030 38 0.333333333333
A000030 39 0.333333333333
A000030 40 0.0
A000030 41 0.333333333333
A000030 42 0.333333333333
A000030 43 0.0
A000030 44 0.0
A000030 45 0.0
A000030 46 0.0
A000030 47 0.333333333333
A000030 48 0.0
A000030 49 0.333333333333
A000030 50 0.0
A000030 51 0.3

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:33: RuntimeWarning: invalid value encountered in true_divide
